In [1]:
# Libary
import dlib
import cv2
import os
import matplotlib.pyplot as plt


from imutils import face_utils

# Libary from progress bar
from tqdm import tqdm

In [2]:
video_path = "/content/for_GO_Asmus_mod.mp4"


In [4]:
def convert_and_trim_bb(image, rect):
	"""
  вычисляем стратовые точки прямоугольника и его ширину с длиной
  """
  
  # extract the starting and ending (x, y)-coordinates of the
	# bounding box
	startX = rect.left()
	startY = rect.top()
	endX = rect.right()
	endY = rect.bottom()
	# ensure the bounding box coordinates fall within the spatial
	# dimensions of the image
	startX = max(0, startX)
	startY = max(0, startY)
	endX = min(endX, image.shape[1])
	endY = min(endY, image.shape[0])
	# compute the width and height of the bounding box
	w = endX - startX
	h = endY - startY
	# return our bounding box coordinates
	return (startX, startY, w, h)


In [3]:
# Модели для обнаружение точек на лице
os.system('wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2')
os.system('bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2')

0

In [4]:
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [40]:
import imageio

# Libary from progress bar
from tqdm import tqdm

def creator_video_from_list(image_list, output_name_video, fps_ = 30, reverse_ = False):
  """
  @image_list -> массив картинок
  @output_name_video -> название выходного видео
  @fps_ -> количество кадров в секунду
  @reverse_ -> включение обратного порядка в видео
  """
  
  writer = imageio.get_writer(output_name_video, fps = fps_)
  if reverse_:
    image_list.reverse()

  for im in tqdm(image_list):
    writer.append_data(im)

  writer.close()
  print('Success create video!!!')

In [7]:
def get_count_cadr_video(path_video):
  """
  Данная функция возвращает количество кадров в видеофайле
  @path_video -> Путь до видеофайла
  """
  
  cnt = 0
  
  cap = cv2.VideoCapture(path_video)
  
  while (cap.isOpened()):
    ret, frame = cap.read()
    if ret:
      cnt += 1
    else:
      break  
  return cnt

cadr = get_count_cadr_video(video_path)
print(cadr)

0


In [52]:
def find_and_print_face_in_image(image):
  
  """
  @image -> image
  """

  detector = dlib.get_frontal_face_detector()
  rects = detector(image)
  ## Return start x,y  and w, h
  boxes = [convert_and_trim_bb(image, r) for r in rects]
  
  color = (0, 255, 0)
  thickness = 2

  for (x, y, w, h) in boxes:
    cv2.rectangle(image, (x, y), (x + w, y + h), color, thickness)      
    #face = image_rgb[y:y + h, x:x + w]
    #plt.imshow(face)
  
  #return first box
  x1 = 0
  x_w = 0
  y1 = 0
  y_h = 0
  for (x, y, w, h) in boxes:
    x1 = x
    x_w = w
    y1 = y
    y_h = h
    break

  return x1,x_w, y1, y_h    
  #plt.imshow(image)

In [30]:
def find_and_print_point_face(image):
  # конвертируем изображение в много оттенков серого
  img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  
  # вычисляем список рамок на каждое найденное лицо
  rects = detector(img_gray, 0)
  
  # вычисляем ключевые точки
  shape = predictor(img_gray, rects[0])
        
  shape = face_utils.shape_to_np(shape)

  radius = 1
  color = (0, 0, 255)
  img_tmp = image
  
  for x, y in shape:
    cv2.circle(img_tmp, (x, y), radius, color , -1)


In [50]:
import os
os.system("pip install ffmpeg-python")
import ffmpeg

def add_audio(video_path, audio_path, name):
  input = ffmpeg.input(audio_path)
  audio = input.audio

  input2 = ffmpeg.input(video_path)
  video = input2.video
  
  out = ffmpeg.output(audio, video, name)
  out.run()


In [67]:
def add_face_in_video(video_path, new_name):
  
  cap = cv2.VideoCapture(video_path)

  frame_width = int(cap.get(3))   #1216

  frame_height = int(cap.get(4))  #2160


  fps = int(cap.get(5))           # 25

  count_cadr = get_count_cadr_video(video_path)

  #### READ VIDEO ####

  cnt = 0

  x1 = 0
  x_w = 0
  y1 = 0
  y_h = 0

  images = []
  for i in tqdm(range(count_cadr)):

      if (cap.isOpened() == False):
        break
    
      # capture each frame of the video
      # frame -> img in BGR color

      ret, frame = cap.read()
   
      if ret:

          ### Перевод изображения в RGB
          image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
          ### face in image ###
          if cnt == 0:
            x1, x_w, y1, y_h = find_and_print_face_in_image(image_rgb)
          else:
              find_and_print_face_in_image(image_rgb)
          

          ### face point
          find_and_print_point_face(image_rgb)

          delta = 100
          sub_img = image_rgb[y1 - delta : y1 + y_h + delta, x1 - delta: x1+ x_w + delta]
          #plt.imshow(sub_img)
          

          images.append(sub_img)
          cnt += 1
      else:
        break    

  print(cnt)
  creator_video_from_list(images, new_name, fps_ = fps )
  add_audio(new_name, video_path, 'a_' + new_name)



In [68]:
add_face_in_video("/content/for_GO_Asmus_mod.mp4", "new_video2.mp4")

100%|██████████| 347/347 [08:08<00:00,  1.41s/it]


347


100%|██████████| 347/347 [00:03<00:00, 86.82it/s]


Success create video!!!
